<a href="https://colab.research.google.com/github/euneuneun/GITGITGIT/blob/main/yolov8_wild_boar_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
os.environ['LANGUAGE'] = 'en_US.UTF-8'

In [ ]:
!pip install ultralytics

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import cv2
from ultralytics import YOLO

In [ ]:
import os,sys
from google.colab import drive ## 구글 콜랩에서 구글 드라이브를 마운트하는 데 사용

drive.mount('/content/gdrive') ## 구글 드라이브를 /content/gdrive 경로에 마운트

!ln -s /content/gdrive/My\ Drive/ /mydrive ## /content/gdrive/My Drive/ 경로를 /mydrive에 심볼릭 링크로 만듬
!ls /mydrive ## /mydrive에 있는 파일 및 폴더를 나열.

!mkdir "/mydrive/ultra_workdir" ## /mydrive에 ultra_workdir이라는 이름의 새 디렉토리를 만듬

In [ ]:
!cd /content/data2; unzip Project2.v2i.yolov8.zip

In [ ]:
model = YOLO("yolov8n.pt")

model.train(data = "/content/data2/data.yaml",epochs = 15, imgsz = 640, project = "/content/gdrive/MyDrive/ultra_workdir",name = "wild", exist_ok = True)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
model = YOLO("/content/gdrive/MyDrive/ultra_workdir/wild/weights/best.pt")

results = model.predict("/content/data2/test/images/1--281-_jpg.rf.2b08f0c842781c8edaeb638623d23091.jpg",line_thickness = 2)

res_plot = results[0].plot()

plt.imshow(res_plot)
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

print(torch.__version__)

In [ ]:
import matplotlib.pyplot as plt

# 결과 딕셔너리
results_dict = {
    'metrics/precision(B)': 0.4347980114478331,
    'metrics/recall(B)': 0.23247232472324722,
    'metrics/mAP50(B)': 0.19870567434925188,
    'metrics/mAP50-95(B)': 0.07180046737975773,
    'fitness': 0.08449098807670714
}

# 성능 지표 키와 값을 추출
metrics = list(results_dict.keys())
values = list(results_dict.values())

# 바 그래프로 시각화
plt.figure(figsize=(10, 6))
plt.bar(metrics, values, color='skyblue')
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.title('Object Detection Metrics')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 결과 데이터
precision = 0.973
recall = 0.417
mAP50 = 0.464
mAP50_95 = 0.328

# 시각화
categories = ['Box(Precision)', 'Box(Recall)', 'mAP50', 'mAP50-95']
scores = [precision, recall, mAP50, mAP50_95]

plt.figure(figsize=(10, 6))
plt.bar(categories, scores, color=['blue', 'green', 'orange', 'red'])
plt.title('Object Detection Performance')
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)  # 점수 범위 (0~1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 결과 데이터
precision = 0.9605483588413889
recall = 0.36458
mAP50 = 0.4448137
mAP50_95 = 0.303352245

# 시각화
categories = ['Box(Precision)', 'Box(Recall)', 'mAP50', 'mAP50-95']
scores = [precision, recall, mAP50, mAP50_95]

plt.figure(figsize=(10, 6))
plt.bar(categories, scores, color=['blue', 'green', 'orange', 'red'])
plt.title('Object Detection Performance')
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)  # 점수 범위 (0~1)
plt.show()

In [ ]:
import cv2
from PIL import Image
from IPython.display import display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np

# YOLO 모델 로드
model = YOLO("/content/gdrive/MyDrive/ultra_workdir/wild/weights/best.pt")

# 웹캠 비디오 스트리밍을 위한 JavaScript 코드
def start_webcam():
    js = """
    var video = document.createElement('video');
    document.body.appendChild(video);
    video.style.display = 'none';

    navigator.mediaDevices.getUserMedia({ video: true }).then(stream => {
        video.srcObject = stream;
        video.play();
    });

    var canvas = document.createElement('canvas');
    canvas.width = 640;
    canvas.height = 480;
    document.body.appendChild(canvas);
    const fps = 10;

    function capture() {
        setTimeout(() => {
            canvas.getContext('2d').drawImage(video, 0, 0, canvas.width, canvas.height);
            var img = canvas.toDataURL('image/jpeg', 0.8);
            google.colab.kernel.invokeFunction('notebook.run_object_detection', [img], {});
            capture();
        }, 1000 / fps);
    }

    capture();
    """

    display(eval_js(js))

# 웹캠에서 캡처된 이미지를 처리하고 객체 감지 결과를 표시하는 함수
def run_object_detection(img_data):
    # 이미지를 OpenCV 형식으로 디코딩
    img = cv2.imdecode(np.frombuffer(b64decode(img_data.split(',')[1]), np.uint8), -1)

    # 객체 감지
    results = model.predict(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))

    # 결과를 OpenCV 이미지로 변환하여 표시
    for result in results.xyxy[0]:
        label = int(result[-1])
        cv2.rectangle(img, (result[0], result[1]), (result[2], result[3]), (0, 255, 0), 2)
        cv2.putText(img, f'Object {label}', (result[0], result[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Object Detection', img)
    cv2.waitKey(1)

# 웹캠 시작
start_webcam()


In [ ]:
from google.colab import files
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# 업로드한 동영상 파일 경로
video_path = "/content/video_01.mp4"

# YOLO 모델 로드
model = YOLO("/content/gdrive/MyDrive/ultra_workdir/wild/weights/best.pt")

# 동영상 열기
cap = cv2.VideoCapture(video_path)

# 동영상 프레임 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 객체 감지
    results = model.predict(pil_img)

    # 결과가 이미지를 렌더링할 수 있는지 확인
    if hasattr(results, 'render'):
        # 결과를 OpenCV 이미지로 변환하여 표시
        frame_with_results = cv2.cvtColor(results.render(), cv2.COLOR_RGB2BGR)
        plt.imshow(frame_with_results)
        plt.axis('off')
        plt.show()

# 종료
cap.release()

In [ ]:
from google.colab import files
import cv2
from PIL import Image
from IPython.display import display

# 업로드한 동영상 파일 경로
video_path = "/content/data2/test/video_01.mp4"

# 업로드한 동영상을 읽기 위해 파일 업로드
uploaded = files.upload()

# 동영상을 열어서 객체 감지
cap = cv2.VideoCapture(video_path)

# YOLO 모델 로드
model = YOLO("/content/gdrive/MyDrive/ultra_workdir/wild/weights/best.pt")

# 동영상의 각 프레임을 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # PIL 이미지로 변환
    pil_img = Image.fromarray(frame)

    # 객체 감지
    results = model.predict(pil_img)

    # 결과를 OpenCV 이미지로 변환하여 출력
    frame_with_results = cv2.cvtColor(results.render(), cv2.COLOR_RGB2BGR)
    cv2.imshow('Object Detection', frame_with_results)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 동영상의 각 프레임을 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # PIL 이미지로 변환
    pil_img = Image.fromarray(frame)

    # 객체 감지
    results = model.predict(pil_img)

    # results가 리스트가 아닌 객체여야 함
    # 객체가 이미지를 렌더링할 수 있는 메서드를 제공하는지 확인
    if hasattr(results, 'render'):
        # 결과를 OpenCV 이미지로 변환하여 출력
        frame_with_results = cv2.cvtColor(results.render(), cv2.COLOR_RGB2BGR)
        cv2.imshow('Object Detection', frame_with_results)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 종료
cap.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt

# 성능 지표 및 결과 딕셔너리
metrics = ['Box(Precision)', 'Box(Recall)', 'mAP50', 'mAP50-95']
results_dict = {'Box(Precision)': 0.961, 'Box(Recall)': 0.365, 'mAP50': 0.445, 'mAP50-95': 0.303}

# 성능 지표에 따른 결과 값 가져오기
values = [results_dict[metric] for metric in metrics]

# 그래프 생성
plt.figure(figsize=(10, 5))
plt.bar(metrics, values, color=['blue', 'green', 'red', 'orange'])
plt.title('YOLO Model Performance')
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.show()

In [ ]:
!cd /content/data2; unzip WILDBOAR.v1i.yolov8.zip